In [4]:
from backtesting import Backtest
import inspect
import pandas as pd
import numpy as np
import seaborn as sns
from itertools import combinations
import tensorflow as tf
import scipy.stats as ss
import strats, get_data
from typing import Tuple
import time
from sklearn.model_selection import BaseCrossValidator
import matplotlib.pyplot as plt
from datetime import date, datetime
import base64
from io import BytesIO

In [5]:
stock_obj = get_data.yFinData("SPY")
try:
    ydata = stock_obj.get_ohlcv()
except:
    print('Uable to download data.')

In [6]:
def get_back_test_comparasion(ydata: pd.DataFrame, strategy: str, data_range, strategy_params: dict,
                              cash: int=1_000_000, commission: float=0.):
    """
    backtest vs buy/hold strategy in strats.py
    input: stock OHLCV dataframe
    output: dataframe of strategy returns, dictionary of trades and equity curve
    """
    avail_strats = [obj for name, obj in inspect.getmembers(strats, inspect.isclass)
                    if (obj.__name__ == strategy) or (obj.__name__ == "BuyAndHold")]
    if not data_range.isdecimal():
        corresponding = {"6m":0.5, "1y":1., "2y":2.}
        data = ydata.iloc[-int(float(corresponding[data_range])*252):]
    else:
        data = ydata.loc["{}-12-31".format(int(data_range)-1):"{}-12-31".format(int(data_range)),]

    temp = []
    sname_temp = []
    equity_trades = {}
    for s in avail_strats:
        if data.shape[0] == 0:
            continue

        bt = Backtest(data, s, cash=cash, commission=commission)
        if s.__name__ == 'SmaCross':
            stats = bt.run(slow = strategy_params['sma_slow'],
                           fast = strategy_params['sma_fast'],
                           long_only = strategy_params['long_only'])
        elif s.__name__ == 'MacdSignal':
            stats = bt.run(fastperiod = strategy_params['fast_period'],
                           slowperiod = strategy_params['slow_period'],
                           signalperiod = strategy_params['signal_period'],
                           long_only = strategy_params['long_only'])
        elif s.__name__ == 'StochOsci':
            stats = bt.run(fastk_period = strategy_params['fast_k_period'],
                           slowk_period = strategy_params['slow_k_period'],
                           slowd_period = strategy_params['slow_d_period'],
                           overbought = strategy_params['overbought'],
                           oversold = strategy_params['oversold'],
                           long_only = strategy_params['long_only'])
        elif s.__name__ == 'StochRsi':
            stats = bt.run(timeperiod = strategy_params['time_period'],
                           fastk_period = strategy_params['fast_k_period'],
                           slowk_period = strategy_params['slow_k_period'],
                           slowd_period = strategy_params['slow_d_period'],
                           overbought = strategy_params['overbought'],
                           oversold = strategy_params['oversold'],
                           long_only = strategy_params['long_only'])
        else:
            stats = bt.run()
        sname = str(stats["_strategy"])
        sname_temp.append("{}_{}".format(sname, data_range))
        temp.append(stats[:27])
        equity_trades["{}_{}".format(sname, data_range)] = (stats["_equity_curve"], stats["_trades"])

    strat_returns = pd.concat(temp, axis=1)
    strat_returns.columns = sname_temp
    return strat_returns

In [7]:
df = get_back_test_comparasion(ydata, "SmaCross", "2y",
                               {"sma_slow":5, "sma_fast":10, "long_only":15}, 1_000_000, 0.)

In [8]:
df

,BuyAndHold_2y,"SmaCross(slow=5,fast=10,long_only=15)_2y"
Start,2019-03-28 00:00:00,2019-03-28 00:00:00
End,2021-03-26 00:00:00,2021-03-26 00:00:00
Duration,729 days 00:00:00,729 days 00:00:00
Exposure Time [%],99.6032,37.6984
Equity Final [$],1.44196e+06,1.03681e+06
Equity Peak [$],1.44196e+06,1.04086e+06
Return [%],44.196,3.68118
Buy & Hold Return [%],46.2545,46.2545
Return (Ann.) [%],20.0816,1.82395
Volatility (Ann.) [%],30.9627,21.7049
